In [4]:
# !pip install simpletransformers
import torch
import pandas as pd
from simpletransformers.classification import ClassificationModel, ClassificationArgs

model_args = ClassificationArgs()
model_args.overwrite_output_dir = True

# Getting two HateBERT models
olid_model = ClassificationModel(
    'bert',
    'GroNLP/hateBERT',
    args=model_args,
    use_cuda=False
)

hasoc_model = ClassificationModel(
    'bert',
    'GroNLP/hateBERT',
    args=model_args,
    use_cuda=False
)

Some weights of the model checkpoint at GroNLP/hateBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/hateBERT and are newly

In [ ]:
# Loading our data in
olid_train = pd.read_csv('data/olid-train-small.csv')
hasoc_train = pd.read_csv('data/hasoc-train.csv')
test = pd.read_csv('data/olid-test.csv')

In [ ]:
# First we train on OLID (IN-DOMAIN)
olid_model.train_model(olid_train, overwrite_output_dir=True)

  0%|          | 0/5852 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/732 [00:00<?, ?it/s]

(732, 0.5484000711493153)

In [ ]:
# Getting the predictions
olid_predictions, _ = olid_model.predict(list(test['text']))

  0%|          | 0/860 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

In [ ]:
# Then we train with HASOC (CROSS-DOMAIN)
hasoc_model.train_model(hasoc_train, overwrite_output_dir=True)

  0%|          | 0/5852 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/732 [00:00<?, ?it/s]

(732, 0.6353497114337858)

In [ ]:
# Getting the predictions
hasoc_predictions, _ = hasoc_model.predict(list(test['text']))

  0%|          | 0/860 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

In [ ]:
# Creatig copies of the test set in order to store our predtictions for future use
from copy import deepcopy

olid_final = deepcopy(test)
hasoc_final = deepcopy(test)

In [ ]:
# Writing our predictions to each dataset respectively
olid_final['predictionns'] = olid_predictions
hasoc_final['predictions'] = hasoc_predictions

In [ ]:
# Writing our predicted sets to disk for analysis later
olid_final.to_csv('in_domain_hateBERT.csv', index=False)
hasoc_final.to_csv('cross_domain_hateBERT.csv', index=False)

In [ ]:
# Here is what we wrote to disk
pd.read_csv('in_domain_hateBERT.csv').head()

,id,text,labels,predictionns
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,1,0
1,27014,"#ConstitutionDay is revered by Conservatives, ...",0,0
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,0,0
3,13876,#Watching #Boomer getting the news that she is...,0,0
4,60133,#NoPasaran: Unity demo to oppose the far-right...,1,0


In [ ]:
# Here is what we wrote to disk
pd.read_csv('cross_domain_hateBERT.csv').head()

,id,text,labels,predictions
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,1,0
1,27014,"#ConstitutionDay is revered by Conservatives, ...",0,0
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,0,0
3,13876,#Watching #Boomer getting the news that she is...,0,0
4,60133,#NoPasaran: Unity demo to oppose the far-right...,1,0
